In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="True"
os.environ["LANGCHAIN_PROJECT"] = os.getenv("LANGCHAIN_PROJECT")

In [2]:
# Data Ingestion
from langchain_community.document_loaders import WebBaseLoader

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [3]:
loader = WebBaseLoader("https://blog.langchain.dev/why-langgraph-platform/")
loader

In [5]:
docs = loader.load()
docs

[Document(metadata={'source': 'https://blog.langchain.dev/why-langgraph-platform/', 'title': 'Why do I need LangGraph Platform for agent deployment?', 'description': 'This blog dives into technical details for why agent deployment is difficult, and how we built a platform to solve those challenges (LangGraph Platform).', 'language': 'en'}, page_content='\n\n\nWhy do I need LangGraph Platform for agent deployment?\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCase Studies\n\n\n\n\nIn the Loop\n\n\n\n\nLangChain\n\n\n\n\nDocs\n\n\n\n\nChangelog\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nWhy do I need LangGraph Platform for agent deployment?\nThis blog dives into technical details for why agent deployment is difficult, and how we built a platform to solve those challenges (LangGraph Platform).\n\n4 min read\nMay 22, 2025\n\n\n\n\n\nLast we week, we announced th

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
final_doc = text_splitter.split_documents(docs)

In [7]:
final_doc

[Document(metadata={'source': 'https://blog.langchain.dev/why-langgraph-platform/', 'title': 'Why do I need LangGraph Platform for agent deployment?', 'description': 'This blog dives into technical details for why agent deployment is difficult, and how we built a platform to solve those challenges (LangGraph Platform).', 'language': 'en'}, page_content='Why do I need LangGraph Platform for agent deployment?\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nCase Studies\n\n\n\n\nIn the Loop\n\n\n\n\nLangChain\n\n\n\n\nDocs\n\n\n\n\nChangelog\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nWhy do I need LangGraph Platform for agent deployment?\nThis blog dives into technical details for why agent deployment is difficult, and how we built a platform to solve those challenges (LangGraph Platform).\n\n4 min read\nMay 22, 2025'),
 Document(metadata={'source': 'https://blog

In [9]:
from langchain.embeddings import SentenceTransformerEmbeddings
embeddings = SentenceTransformerEmbeddings()



C:\Users\aakas\AppData\Local\Temp\ipykernel_21688\470793425.py:2: LangChainDeprecationWarning: Default values for HuggingFaceEmbeddings.model_name were deprecated in LangChain 0.2.16 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceEmbeddings constructor instead.
  embeddings = SentenceTransformerEmbeddings()


In [10]:
from langchain_community.vectorstores import FAISS
vectordb = FAISS.from_documents(final_doc,embeddings)

In [ ]:
vectordb

In [12]:
query = "The longer an agent runs, the longer it is likely to suffer an exception"
result = vectordb.similarity_search(query)
result[0].page_content

'prolonged processes. Additionally, stream endpoints can be rejoined if the connection drops, and will very soon buffer events emitted while you were disconnected.ProblemThe longer an agent runs for, the more likely it is to suffer an exception while running.How we solve itLangGraph Server implements strategies to both minimize the number of exceptions agents face (eg. by running them in workers which can use isolated event loops and background threads) as well as the amount of time needed to recover from an exception (eg. when an agent fails it is retried a configurable number of times, and each retry starts off from the most recent successful checkpoint).ProblemThe longer an agent runs for, the more you need intermediate output to be emitted while the agent is running, to show the end user that something is happening.How we solve itLangGraph Server implements multiple streaming modes, including intermediate results, token-by-token LLM messages, all the way to streaming of custom'

In [13]:
# Retrieval Chain,Document chain
import os
from langchain_groq import ChatGroq
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template(
    """
Answer the following question based only on the provided context
<context>
{context}
</context>
"""
)
llm = ChatGroq(api_key=os.environ["GROQ_API_KEY"],model="llama-3.3-70b-versatile",temperature=0.5)
document_chain = create_stuff_documents_chain(llm,prompt)
document_chain

RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableLambda(format_docs)
}), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
| ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000028028835A30>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000028049621190>, model_name='llama-3.3-70b-versatile', temperature=0.5, model_kwargs={}, groq_api_key=SecretStr('**********'))
| StrOutputParser(), kwargs={}, config={'run_name': 'stuff_documents_chain'}, config_factories=[])

In [16]:
document_chain.invoke(
    {
        "input":"The longer an agent runs, the longer it is likely to suffer an exception",
        "context":final_doc
    }
)

'You need the LangGraph Platform for agent deployment because it solves the challenges associated with deploying long-running, stateful, or bursty agents. These challenges include:\n\n1. Maintaining open connections for long-running agents\n2. Handling timeouts and disruptions for prolonged processes\n3. Minimizing exceptions and recovering from them\n4. Emitting intermediate output for long-running agents\n5. Handling bursty request loads\n6. Managing stateful agents with short-term and long-term memory\n7. Implementing human-in-the-loop and human-on-the-loop interactions\n8. Managing memory usage for conversations and memories\n\nThe LangGraph Platform provides solutions to these challenges, including:\n\n1. Launching agent runs in the background with polling endpoints, streaming endpoints, and webhooks\n2. Sending regular heartbeat signals to prevent connection closures\n3. Implementing strategies to minimize exceptions and recover from them\n4. Providing multiple streaming modes fo

In [18]:
# Retriever

retriever = vectordb.as_retriever()

In [19]:
from langchain.chains import create_retrieval_chain

retrieval_chain = create_retrieval_chain(retriever,document_chain)

In [20]:
retrieval_chain

RunnableBinding(bound=RunnableAssign(mapper={
  context: RunnableBinding(bound=RunnableLambda(lambda x: x['input'])
           | VectorStoreRetriever(tags=['FAISS', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x0000028028834830>, search_kwargs={}), kwargs={}, config={'run_name': 'retrieve_documents'}, config_factories=[])
})
| RunnableAssign(mapper={
    answer: RunnableBinding(bound=RunnableBinding(bound=RunnableAssign(mapper={
              context: RunnableLambda(format_docs)
            }), kwargs={}, config={'run_name': 'format_inputs'}, config_factories=[])
            | ChatPromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='\nAnswer the following question based only on the provided context\n<context>\n{context}\n</context>\n'), additional_kwargs={})])
            | 

In [24]:
#  Get the response from the LLM

response = retrieval_chain.invoke({"input":".What to do to reduce exceptions?"})
response['answer']

"I'm ready to answer your question based on the provided context. What is your question?"